In [1]:
%cd ..

/data/kazanplova/latent_vae_upscale_train


In [2]:
import pandas as pd


df = pd.read_csv('clear_images_with_maniqa_scores.csv')

In [3]:
# from pathlib import Path
# import shutil

# output_root = Path("best_samples")
# output_root.mkdir(parents=True, exist_ok=True)

# df_with_avg = df.assign(clip_hyper_avg=(df["clip_iqa"] + df["hyperiqa"]) / 2)
# targets = {
#     "worst_iqa": df_with_avg.nlargest(200, "clip_iqa"),
#     "worst_hyperiqa": df_with_avg.nlargest(200, "hyperiqa"),
#     "worst_clip_hyper_avg": df_with_avg.nlargest(200, "clip_hyper_avg"),
# }

# def copy_subset(dest_name: str, subset: pd.DataFrame) -> None:
#     dest_dir = output_root / dest_name
#     dest_dir.mkdir(parents=True, exist_ok=True)
#     for _, row in subset.iterrows():
#         src_path = Path(row["path"]).expanduser()
#         if not src_path.exists():
#             print(f"Skipping missing file: {src_path}")
#             continue
#         dest_path = dest_dir / src_path.name
#         if dest_path.exists():
#             dest_path = dest_dir / f"{src_path.stem}_{row.name}{src_path.suffix}"
#         shutil.copy(src_path, dest_path)
#     print(f"Copied {len(subset)} images to {dest_dir}")

# for name, subset in targets.items():
#     limited_subset = subset.head(min(200, len(subset)))
#     copy_subset(name, limited_subset)

In [4]:
# !rm -rf best_samples
# !rm -rf worst_samples

In [5]:
df_with_avg = df.assign(clip_hyper_avg=(df["clip_iqa"] + df["hyperiqa"]) / 2)
count_num = 100000
quality_df = pd.concat([
    df_with_avg.nlargest(100000, "clip_iqa"),
    df_with_avg.nlargest(100000, "hyperiqa"),
    df_with_avg.nlargest(100000, "clip_hyper_avg"),
], axis=0)
quality_df.drop_duplicates(inplace=True)
quality_df

,stage,path,size_bytes,width,height,over_limit,available_embeddings,clip_iqa,hyperiqa,clip_hyper_avg
6666,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,2168022.0,1152.0,864.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.849434,0.261990,0.555712
38175,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,1420922.0,1152.0,648.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.824737,0.428320,0.626529
677070,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,433017.0,1920.0,1280.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.823075,0.417921,0.620498
579253,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,761276.0,2592.0,1728.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.820682,0.404958,0.612820
53239,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,1214512.0,1152.0,648.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.819316,0.402350,0.610833
...,...,...,...,...,...,...,...,...,...,...
103558,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,1670892.0,1152.0,720.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.621246,0.367549,0.494397
550326,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,40614.0,265.0,400.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.619190,0.369601,0.494396
468225,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,43566.0,265.0,400.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.620040,0.368750,0.494395
7872,validate,/data/kazanplova/latent_vae_upscale_train/unpa...,573177.0,1152.0,720.0,NaN,['/data/kazanplova/latent_vae_upscale_train/un...,0.611390,0.377385,0.494388


In [6]:
quality_df = quality_df.sample(n=10)

In [7]:
import os
import shutil
import warnings
from multiprocessing import Pool, cpu_count
from pathlib import Path
from typing import Any, Iterable, Iterator, List, Optional, Tuple, Union, Set

from PIL import Image
from tqdm.auto import tqdm

embeddings_root = "/data/kazanplova/latent_vae_upscale_train/unpacked_original_ds/full_dataset/cache_vae_embeddings"

# ---------- helpers ----------
def _rel_after_cache_root(p: Path) -> Path:
    """Return path segment after 'cache_vae_embeddings/'."""
    parts = p.parts
    if "cache_vae_embeddings" not in parts:
        raise ValueError(f"'cache_vae_embeddings' not in: {p}")
    return Path(*parts[parts.index("cache_vae_embeddings") + 1:])

def _new_size(w: int, h: int, long_side: int) -> Tuple[int, int]:
    L = max(w, h)
    if L <= long_side:
        return w, h
    s = long_side / float(L)
    return max(1, int(round(w * s))), max(1, int(round(h * s)))

def _resize_task(job: Tuple[str, str, int]) -> Tuple[str, Optional[str]]:
    """
    Worker task. Returns (src_path, error_msg_or_None).
    Special case: if a palette image has per-palette (byte) transparency,
    we skip it and return a 'skipped:' message.
    """
    src, dst, long_side = job
    try:
        with warnings.catch_warnings():
            # Turn ONLY the specific Pillow warning into an error
            warnings.filterwarnings(
                "error",
                category=UserWarning,
                message=r"Palette images with Transparency expressed in bytes.*",
            )
            with Image.open(src) as im:
                # If the above warning would have been raised (i.e., such an image),
                # it is now an exception we catch below and mark as skipped.
                im = im.convert("RGB")
                ns = _new_size(*im.size, long_side)
                if ns != im.size:
                    im = im.resize(ns, Image.LANCZOS)
                Path(dst).parent.mkdir(parents=True, exist_ok=True)
                im.save(dst)
        return src, None

    except UserWarning:
        # Explicitly skip these images; don't write anything to dst
        return src, "skipped: palette transparency (bytes)"

    except Exception as e:
        return src, str(e)

def _run_resize(
    jobs: List[Tuple[str, str, int]],
    workers: Optional[int],
    verbose: bool
) -> Tuple[int, int, int]:
    """Run resize jobs in parallel. Returns (ok_count, err_count, skipped_count)."""
    if not jobs:
        return 0, 0, 0

    n = max(1, min(workers or cpu_count(), cpu_count()))
    ok = err = skipped = 0

    if verbose:
        print(f"Resizing {len(jobs)} images to max {jobs[0][2]}px with {n} worker(s)...")

    with Pool(processes=n) as pool, tqdm(total=len(jobs), desc="Resizing images", leave=False) as pbar:
        for _, msg in pool.imap_unordered(_resize_task, jobs):
            if msg is None:
                ok += 1
            elif isinstance(msg, str) and msg.startswith("skipped:"):
                skipped += 1
            else:
                err += 1
            pbar.update(1)

    if verbose:
        print(f"Resized {ok} images; skipped {skipped} (palette byte transparency); {err} failures.")

    return ok, err, skipped

def _copy_embedding_task(job: Tuple[str, str, bool]) -> Tuple[str, Optional[str]]:
    """Copy embeddings in parallel, returning (src_path, error_or_None)."""
    src, dst, overwrite = job
    try:
        dst_path = Path(dst)
        if dst_path.exists() and not overwrite:
            return src, "skipped"
        dst_path.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst_path)
        return src, None
    except Exception as exc:
        return src, str(exc)

# ---------- new helpers for nested inputs ----------
def _iter_paths_maybe_nested(items: Iterable[Any]) -> Iterator[Path]:
    """
    Yield Path objects from a possibly nested iterable of paths (Path/str) or
    iterables thereof (list/tuple/set). Non-path scalars are ignored.
    """
    for x in items:
        if isinstance(x, (list, tuple, set)):
            # Recurse into nested iterables
            yield from _iter_paths_maybe_nested(x)
        else:
            # Accept Path-like (Path/str); skip anything else quietly
            if isinstance(x, (Path, str)):
                yield Path(x)

def _filter_by_model(paths: Iterable[Path], model_names: Optional[List[str]]) -> Iterator[Path]:
    """Yield only paths that contain any of the given model name substrings (if provided)."""
    if not model_names:
        yield from paths
        return
    for p in paths:
        s = str(p)
        if any(m in s for m in model_names):
            yield p

# ---------- main ----------
def process_images_and_embeddings(
    image_paths: Iterable[Union[Path, str, Iterable[Any]]],
    embedding_paths: Iterable[Union[Path, str, Iterable[Any]]],
    destination_root: Path,
    *,
    overwrite: bool = False,
    verbose: bool = True,
    resize_long_side: int = 1024,
    resize_dir_name: str = "resized_1024",
    resize_workers: Optional[int] = None,
    embedding_workers: Optional[int] = None,
    model_names : Optional[List[str]] = None,
) -> None:
    """
    - DOES NOT copy original images.
    - Resizes images into `destination_root / resize_dir_name / ...`
    - Copies embeddings by rewriting their prefix to `destination_root/cache_vae_embeddings/...`
      using the segment after 'cache_vae_embeddings/'.
    - Skips palette PNGs with transparency expressed in bytes (Pillow's specific UserWarning).
    - Copies embeddings in parallel using multiprocessing.
    - Supports nested (list/tuple/set) inputs for image_paths and embedding_paths.
    """
    # normalize images (allow nested too, harmless if flat)
    imgs = [Path(p) for p in tqdm(_iter_paths_maybe_nested(image_paths))]

    # normalize + (optionally) filter embeddings — now supports nested input
    flat_embs_iter = _iter_paths_maybe_nested(embedding_paths)
    embs = list(_filter_by_model(flat_embs_iter, model_names))

    if not imgs and not embs:
        if verbose:
            print("Nothing to do.")
        return

    if resize_long_side <= 0:
        raise ValueError("resize_long_side must be > 0")

    dest = Path(destination_root).expanduser().resolve()
    dest.mkdir(parents=True, exist_ok=True)

    # --- images: build jobs + resize ---
    img_ok = img_err = img_skipped = 0
    if imgs:
        dataset_root = Path(os.path.commonpath([str(p.parent) for p in imgs]))
        out_root = dest / resize_dir_name
        out_root.mkdir(parents=True, exist_ok=True)

        jobs: List[Tuple[str, str, int]] = []
        for p in tqdm(imgs, desc="Preparing resize jobs", leave=False):
            rel = p.relative_to(dataset_root)
            dst = out_root / rel
            if dst.exists() and not overwrite:
                continue
            jobs.append((str(p), str(dst), int(resize_long_side)))

        img_ok, img_err, img_skipped = _run_resize(jobs, resize_workers, verbose)

    # --- embeddings: parallel copy via prefix rewrite (refactored) ---
    copied = skipped = failed = 0
    copy_jobs: List[Tuple[str, str, bool]] = []
    seen_destinations: Set[Path] = set()

    for epath in tqdm(embs, desc="Preparing embedding copy jobs", leave=False):
        try:
            rel_path = _rel_after_cache_root(Path(epath))
        except ValueError:
            failed += 1
            continue

        dst_path = dest / "cache_vae_embeddings" / rel_path

        if not overwrite and dst_path in seen_destinations:
            skipped += 1
            continue

        seen_destinations.add(dst_path)
        copy_jobs.append((str(epath), str(dst_path), overwrite))

    if copy_jobs:
        n_workers = max(1, min(embedding_workers or cpu_count(), cpu_count()))
        with Pool(processes=n_workers) as pool, tqdm(total=len(copy_jobs), desc="Copying embeddings", leave=False) as pbar:
            for _, msg in pool.imap_unordered(_copy_embedding_task, copy_jobs, chunksize=32):
                if msg is None:
                    copied += 1
                elif msg == "skipped":
                    skipped += 1
                else:
                    failed += 1
                pbar.update(1)
        if verbose:
            print(f"Copied embeddings with {n_workers} worker(s).")

    # --- reporting ---
    if verbose:
        if imgs:
            print(
                f"Images -> ok: {img_ok}, skipped (palette byte transparency): {img_skipped}, errors: {img_err}. "
                f"Output dir: {dest / resize_dir_name}"
            )
        if embs:
            print(f"Embeddings -> copied: {copied}, skipped: {skipped}, failed: {failed}.")


/home/kazanplova/anaconda3/envs/latent_vae_train/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
!ls

6xA100_config.yaml		     pretrained_weights
LICENSE				     requirements.txt
calculate_maniqa_scores.py	     result_cleared_df.csv
clear_images.csv		     result_cleared_df_1.csv
clear_images.py			     runs
clear_images_with_maniqa_scores.csv  scripts
configs				     test_model.py
connect_embeddings_with_images.py    train_vae.py
data_df.csv			     train_vae_distributed.py
extract_data.py			     training
latent_upscale_quality_170k_1024px   unpacked_original_ds
list_image_sizes.py		     upscale_df_quality_170k
load_data.sh			     wandb
precompute_embeddings.py


In [ ]:
# !rm -rf upscale_df_quality_170k

In [ ]:

new_dir_latent_uspcale_simple_df = "upscale_df_quality_170k"
process_images_and_embeddings(
    image_paths=quality_df["path"],
    embedding_paths=quality_df["available_embeddings"].apply(eval).tolist(),
    destination_root=new_dir_latent_uspcale_simple_df,
    resize_workers=120,
    embedding_workers=40,
    model_names=["flux_vae", "sd3_vae_anime_ft"]
)

100%|██████████| 10/10 [00:00<00:00, 62695.13it/s]           , ?it/s]
                                                                     

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'list'

In [ ]:
# Example usage (uncomment to run):
# sample_paths = quality_df["path"].head(10).tolist()
# copy_images_with_embeddings(sample_paths, "flux_vae", Path("./test_dataset_subset"), manifest=quality_df)